# Numpy

¿Que vamos a ver hoy?

- ND-Array
- dtype
- index/slicing (repaso)
- mascaras e indices booleanos
- operaciones vectoriales (no vamos a ver matematica hoy)
- Reglas para extender arrays en operaciones
- Que es un seed
- idea de ufunc
  - sobre un vector
  - sobre dos vectores
  - agregacion
- reduccion con axis
- leer archivos genfromtxt
- cambiar tipos astype

## Funciones especificas

- array
- shape
- size
- ndim
- arange
- ones
- zeros
- reshape
- random
- random.uniform
- empty


## otras cosas

- operador %

# Problema a solucionar

Tenemos un archivo excel con información sobre inversiones e inflación. Queremos saber que inversiones le ganaron a la inflación y cuales no. 

Vamos a suponer las siguientes cuestiones:

- Lo que se cobra como cupon se descuenta del valor del bono
- La plata que se cobra en cupon o interes se guarda en algun activo que ajusta por inflacion. 

![Inversion](./Inversion.png)

En este problema, tenemos que solucionar los siguientes problemas:

- Leer la información.
- Poder hacer cuentas mas o menos complejas (vamos a ver cuales, no hace falta que las inventen ustedes) financieras repitiendo operaciones para los datos.
- Chequear que la informacion tenga cierto sentido.

## Pasos a realizar

- Leer la informacion correctamente y separar los encabezados del contenido.
- Separar la informacion segun la columna 

- Chequear que la informacion sea valida (a ojo y matematicamente)

- Construir un índice de inflacion a partir de los porcentajes de inflación mensual. 
- Calcular cuanto valdrian las inversiones si se colocara en algun activo que se ajuste por inflación.
- Incluir el ajuste de valor de los cupones e interes. 
- Calcular cuanto valen las inversiones al finalizar el periodo para poder comparar con la inflación, incluyendo los cupones e interes.


In [18]:
# importamos numpy

import numpy as np

In [19]:
filename = "./Datos.csv"

data_cruda = np.genfromtxt(filename, delimiter="\t", dtype = str) # Notar que si no ponemos el separador correcto tira error. Y si no ponemos dtype lee todo como vacio.

data_cruda

array([['', 'Enero', '', '', 'Febrero', '', '', 'Marzo', '', '', 'Abril',
        '', '', 'Mayo', '', '', 'Junio', '', '', 'Julio', '', '',
        'Agosto', '', '', 'Septiembre', '', '', 'Octubre', '', '',
        'Noviembre', '', '', 'Diciembre', '', ''],
       ['', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)', 'Nominales', 'Valor', 'C&I (U)', 'Nominales', 'Valor',
        'C&I (U)'],
       ['Inflacion (%)', '', '2', '', '', '3.1', '', '', '1.9', '', '',
        '1.4', '', '', '1.5', '', '', '1.8', '', '', '2.2', '', '',
        '1.6', '', '', '1.2', '', '', '1', '', '', '0.9', '', '', '1.2',
        ''],
       ['ETH', '1000', '123', '', '', '130', '', '', '145',

In [20]:
# Veamos algunas propiedades de la informacion

data_cruda.size
data_cruda.shape
data_cruda.ndim

# Busquemos alguna informacion util

data_cruda[0]

data_cruda[1]

data_cruda[2]

data_cruda[3:6]

# Si quisieramos hacerlo mas robusto:

lineas_de_encabezado = 2

encabezados = data_cruda[:lineas_de_encabezado]
inflacion = data_cruda[lineas_de_encabezado]
contenido = data_cruda[lineas_de_encabezado + 1:]

# Notar que en la segunda seccion no hay valores arbitrarios, todos los valores se hacen con funciones o con un sentido logico (por ej el +1)

In [21]:
# Vamos a construir un index para extraer los nombres de los meses

# Primero queremos entender como construirlo

encabezados[0]

# Hay que hacer a partir del 1 y uno de cada tres hasta el ultimo

start = 1
stop = encabezados[0].size
step = 3
index = np.arange(start,stop,step)

meses = encabezados[0][index] # Notar la doble indexacion

# Ahora queremos saber en que columnas hay info del valor, de la cantidad de nominales (comprado o vendidos), y del Cupon o interes

# primero inspeccionamos la fila

encabezados[1]

# Los textos son: "Nominales", "Valor" y "C&I (U)"

texto = "Valor"
mascara_indices_valor = encabezados[1] == texto
print (mascara_indices_valor)

texto = "Nominales"
mascara_indices_nominal = encabezados[1] == texto

texto = "C&I (U)"
mascara_indices_cupon = encabezados[1] == texto



[False False  True False False  True False False  True False False  True
 False False  True False False  True False False  True False False  True
 False False  True False False  True False False  True False False  True
 False]


In [22]:
# Ahora vamos a extraer la informacion en forma de diccionario para cada activo

activos = []

# contenido[contenido == ""] = np.nan
contenido[contenido == ""] = "0"

for elemento in contenido:
    print (elemento)
    activo = {}
    activo["Nombre"] = elemento[0]
    activo["Valores"] = elemento[mascara_indices_valor].astype(np.float)
    activo["Cupones"] = elemento[mascara_indices_cupon].astype(np.float)
    activo["Nominales"] = elemento[mascara_indices_nominal].astype(np.float)
    activos.append(activo)

print (activos[0])

['ETH' '1000' '123' '0' '0' '130' '0' '0' '145' '0' '0' '150' '0' '0'
 '110' '0' '0' '105' '0' '0' '132' '0' '-500' '148' '0' '0' '170' '0' '0'
 '185' '0' '0' '156' '0' '-500' '140' '0']
['GOLD' '0' '0' '0' '73' '110' '5' '10' '111' '5' '50' '132' '5' '-80'
 '112' '5' '0' '115' '5' '-53' '109' '5' '0' '85' '20' '0' '68' '20' '0'
 '45' '20' '0' '22' '20' '0' '0' '20']
['Al29' '300' '54' '0' '0' '57' '0' '-150' '60' '0' '0' '62' '0' '200'
 '65' '0' '0' '69' '0' '-150' '75' '0' '0' '72' '7' '-100' '70' '3' '0'
 '73' '7' '-100' '74' '3' '0' '77' '7']
{'Nombre': 'ETH', 'Valores': array([123., 130., 145., 150., 110., 105., 132., 148., 170., 185., 156.,
       140.]), 'Cupones': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Nominales': array([1000.,    0.,    0.,    0.,    0.,    0.,    0., -500.,    0.,
          0.,    0., -500.])}


In [23]:
# Hasta aca con listas se podria haber hecho casi que lo mismo.

# Vamos a calcular la inflacion acumulada desde cada mes hasta el final. 

# Veamos primero como es la info que tenemos de inflacion.

inflacion

inflacion_numerica = inflacion[mascara_indices_valor].astype(np.float)

inflacion_numerica # Esto esta en porcentaje pero si la inflacion fue de 2% quiere decir que hay que multuiplicar el precio inicial por 1.02

# Como hacemos eso de manera linda con vectores numpy

inflacion_numerica = inflacion_numerica /100 + 1 # Esto no se podia hacer tan facil con listas!!!!

inflacion_numerica

inflacion_acumulada = inflacion_numerica.cumprod()

inflacion_acumulada

# Esto sirve si tenemos un valor en Enero y queremos saber cuando va a valer por ejemplo hasta Julio, hacemos el valor en enero por el valor del casillero Julio, pero no sirve si queremos hacer el valor en diciembre de algo que tenemos en Julio
# Para esta segunda cuenta necesitamos multiplicar por la inflacion mensual desde atras. 

inflacion_acumulada_reversa = np.flip(np.flip(inflacion_numerica).cumprod())

inflacion_acumulada_reversa

array([1.21675451, 1.19289658, 1.15702869, 1.13545504, 1.11977815,
       1.1032297 , 1.0837227 , 1.06039403, 1.04369491, 1.03131908,
       1.021108  , 1.012     ])

# Calculo de ganancia por cupones o intereses

Para calcular cuanto no da de intereses o cupones un activo hay que hacer lo siguiente:

1) Saber cuantos nominales tenemos de cada activo cada mes.
2) Multiplicar esos nominales por lo que rinde cada nominal ese mes para saber el monto de beneficio de cada mes.
3) Multiplicar el monto de beneficio de cada mes por el ajuste de inflacion (reverso porque es desde ese mes hasta el final)


In [24]:
activo = activos[0]

print (activo["Nominales"])
print (activo["Cupones"])

# ¿Como hacemos para saber cuantos nominales tenemos mes a mes si sabemos las compras y ventas?

activo["Nominales_totales"] = activo["Nominales"].cumsum()

print (activo["Nominales_totales"])

activo["Beneficio_mes_a_mes"] = activo["Nominales_totales"] * activo["Cupones"]

print (activo["Beneficio_mes_a_mes"])
activo["Beneficio_mes_a_mes_valuado_hoy"] = activo["Beneficio_mes_a_mes"] * inflacion_acumulada_reversa
activo["Beneficio_mes_a_mes_valuado_hoy"]
activo["Beneficio_valuado_hoy_total"] = activo["Beneficio_mes_a_mes_valuado_hoy"].sum()

print (activo["Beneficio_valuado_hoy_total"])

# Podriamos hacer esto para todos los activos en una sola linea, pero lo dejamos aca. 

[1000.    0.    0.    0.    0.    0.    0. -500.    0.    0.    0. -500.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1000. 1000. 1000. 1000. 1000. 1000. 1000.  500.  500.  500.  500.    0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0.0


# Calculo de ganancia por valor o compra - venta de activos

La idea es similar a la anterior pero ahora hay que considerar 

1) El valor al dia de hoy de los gastado en comprar o lo ganado en vender 
2) La cantidad de activos que quedan. Para esto ultimo podemos considerar que vendemos todo lo que queda en el ultimo mes.

In [25]:
print (activo["Valores"])
print (activo["Nominales"])

# Vamos a modificar la lista de nominales para que "se venda" todo en el ultimo mes.

activo["Nominales_liquidado"] = activo["Nominales"]
activo["Nominales_liquidado"][-1] = activo["Nominales_liquidado"][-1] + - activo["Nominales"].sum()
activo["Nominales_liquidado"]

activo["CambioValor_mes_a_mes_valudado_hoy"] = - activo["Nominales_liquidado"]*activo["Valores"]*inflacion_acumulada_reversa # El menos es porque un nominal positivo es comprar pero es un gasto.

print (activo["CambioValor_mes_a_mes_valudado_hoy"])

activo["CambioValor_valuado_hoy"] = activo["CambioValor_mes_a_mes_valudado_hoy"].sum()

print (activo["CambioValor_valuado_hoy"])

print (activo["Beneficio_valuado_hoy_total"])
print ("Ganancia neta valudad hoy: " + str(activo["CambioValor_valuado_hoy"] + activo["Beneficio_valuado_hoy_total"]))


[123. 130. 145. 150. 110. 105. 132. 148. 170. 185. 156. 140.]
[1000.    0.    0.    0.    0.    0.    0. -500.    0.    0.    0. -500.]
[-149660.80487003      -0.              -0.              -0.
      -0.              -0.              -0.           78469.15803525
      -0.              -0.              -0.           70840.        ]
-351.6468347826449
0.0
Ganancia neta valudad hoy: -351.6468347826449


# Pasamos prolijo todo

In [26]:
import numpy as np

filename = "./Datos.csv"

data_cruda = np.genfromtxt(filename, delimiter="\t", dtype = str) # Notar que si no ponemos el separador correcto tira error. Y si no ponemos dtype lee todo como vacio.

lineas_de_encabezado = 2


encabezados = data_cruda[:lineas_de_encabezado]
inflacion = data_cruda[lineas_de_encabezado]
contenido = data_cruda[lineas_de_encabezado + 1:]

# Los textos son: "Nominales", "Valor" y "C&I (U)"

texto = "Valor"
mascara_indices_valor = encabezados[1] == texto
texto = "Nominales"
mascara_indices_nominal = encabezados[1] == texto
texto = "C&I (U)"
mascara_indices_cupon = encabezados[1] == texto

inflacion = inflacion[mascara_indices_valor].astype(np.float)


activos = []
contenido[contenido == ""] = "0"

for elemento in contenido:
    activo = {}
    activo["Nombre"] = elemento[0]
    activo["Valores"] = elemento[mascara_indices_valor].astype(np.float)
    activo["Cupones"] = elemento[mascara_indices_cupon].astype(np.float)
    activo["Nominales"] = elemento[mascara_indices_nominal].astype(np.float)
    activos.append(activo)


In [27]:
def calcular_inflacion_reversa(inflacion):
    inflacion_numerica = inflacion /100 + 1 # Esto no se podia hacer tan facil con listas!!!!
    inflacion_acumulada_reversa = np.flip(np.flip(inflacion_numerica).cumprod())
    return inflacion_acumulada_reversa

In [28]:
# Ahora ponemos todo en una funcion:

def calcular_ganancia_total_activo (activo,inflacion):
    inflacion_reversa= calcular_inflacion_reversa(inflacion)

    activo["Nominales_totales"] = activo["Nominales"].cumsum()
    activo["Beneficio_mes_a_mes"] = activo["Nominales_totales"] * activo["Cupones"]
    activo["Beneficio_mes_a_mes_valuado_hoy"] = activo["Beneficio_mes_a_mes"] * inflacion_reversa
    activo["Beneficio_valuado_hoy_total"] = activo["Beneficio_mes_a_mes_valuado_hoy"].sum()

    activo["Nominales_liquidado"] = activo["Nominales"]
    activo["Nominales_liquidado"][-1] = activo["Nominales_liquidado"][-1] + - activo["Nominales"].sum()
    activo["CambioValor_mes_a_mes_valudado_hoy"] = - activo["Nominales_liquidado"]*activo["Valores"]*inflacion_reversa # El menos es porque un nominal positivo es comprar pero es un gasto.
    activo["CambioValor_valuado_hoy"] = activo["CambioValor_mes_a_mes_valudado_hoy"].sum()

    return activo["CambioValor_valuado_hoy"] + activo["Beneficio_valuado_hoy_total"]


In [29]:
for activo in activos:
    print (f"El activo {activo['Nombre']} reporto una ganancia de ${calcular_ganancia_total_activo(activo,inflacion)}.")

El activo ETH reporto una ganancia de $-351.6468347826449.
El activo GOLD reporto una ganancia de $196.36241294312276.
El activo Al29 reporto una ganancia de $5718.246539360956.
